<a href="https://colab.research.google.com/github/rsthakur8815/CV-Assignment/blob/main/Code_Assignment_3_RamdevLodhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from math import log2

In [ ]:
data = {
    'age': ['<=30', '<=30', '31..40', '>40', '>40', '>40', '31..40', '31..40', '<=30', '<=30', '>40', '<=30', '<=30', '31..40', '31..40', '>40'],
    'income': ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'low', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student': ['no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no'],
    'credit_rating': ['fair', 'fair', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair'],
    'buys_computer': ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
}
df = pd.DataFrame(data)

In [ ]:
def entropy(class_probabilities):
    return sum(-p * log2(p) for p in class_probabilities if p)


In [ ]:

# Function to calculate information gain
def information_gain(data, feature_name, target_name):
    # Calculate total entropy before split
    total_entropy = entropy([len(data[data[target_name] == target]) / len(data) for target in data[target_name].unique()])


    weighted_entropy = 0
    for value in data[feature_name].unique():
        subset = data[data[feature_name] == value]
        weighted_entropy += len(subset) / len(data) * entropy([len(subset[subset[target_name] == target]) / len(subset) for target in subset[target_name].unique()])

    # Calculate information gain
    return total_entropy - weighted_entropy


In [ ]:

# Function to find best feature to split on
def find_best_split(data, target_name):
    best_feature = None
    best_gain = -1
    for feature in data.columns[:-1]:
        gain = information_gain(data, feature, target_name)
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
    return best_feature


In [ ]:

# Define Node class for the decision tree
class Node:
    def __init__(self, feature=None, value=None, result=None):
        self.feature = feature
        self.value = value
        self.result = result
        self.children = {}


In [ ]:


def build_tree(data, target_name):

    if len(data[target_name].unique()) == 1:
        return Node(result=data[target_name].iloc[0])


    if len(data.columns) == 1:
        return Node(result=data[target_name].mode().iloc[0])


    best_feature = find_best_split(data, target_name)


    node = Node(feature=best_feature)


    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        node.children[value] = build_tree(subset.drop(columns=[best_feature]), target_name)

    return node


In [ ]:


def classify(instance, tree):
    if tree.result is not None:
        return tree.result
    value = instance[tree.feature]
    if value not in tree.children:
        return tree.children['<other>'].result
    return classify(instance, tree.children[value])



In [ ]:

tree = build_tree(df, 'buys_computer')

# Test the decision tree
test_instance = {'age': '<=30', 'income': 'medium', 'student': 'yes', 'credit_rating': 'fair'}
print("Prediction for test instance:", classify(test_instance, tree))

Prediction for test instance: yes
